In [ ]:
# Importing necessary libraries
import geopandas as gpd
import rasterio.mask
import fiona
import os
import numpy as np
import shapefile
import rasterstats
import matplotlib.pyplot as plt
import statistics
import pandas as pd
import rasterio
import time
import cv2
import math

In [ ]:
from typing import cast, Optional

# Importing necessary programs from the libraries
from rasterio.plot import show
from rasterio.transform import from_origin
from rasterio.crs import CRS
from rasterio import features
from rasterstats import zonal_stats
from matplotlib.colors import ListedColormap
from matplotlib import pyplot as plt

In [ ]:
import os

# helper funcs

# def display_image(img: np.ndarray, image_name: str, *, dir: str = "./Test_Images"):
#     """expects array input in BGR"""
#     img_vals = (img * 255).astype(np.uint8)
#     file_path = os.path.join(dir, f"{image_name}.jpg")
#     cv2.imwrite(file_path, img_vals)
#     display(Image(filename=file_path))

import PIL.Image

def display_image(img: np.ndarray, _: Optional[str] = None):
    display(PIL.Image.fromarray(cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_BGR2RGB)))

def show_contours(img: np.ndarray, contours):
    img_with_contours = img.copy()
    for contour in contours:
        cv2.drawContours(img_with_contours, [contour], 0, (0, 1, 0), 2)
    display_image(img_with_contours)

In [ ]:
FILENAME = "Test_Images/Test_Image_1.JPG"
IMAGE_BITS = 8

## Image Setup

In [ ]:
image_max_value: int
red_channel = math.inf
green_channel = math.inf
blue_channel = math.inf
nir_channel = math.inf
re_channel = math.inf

_, file_extension = os.path.splitext(FILENAME)
file_extension = file_extension[1:].lower()
if file_extension in {"tif", "tiff"}:
    # red_channel = 3
    # green_channel = 2
    # blue_channel = 1
    # nir_channel = 4
    # re_channel = 5

    red_channel = 1
    green_channel = 2
    blue_channel = 3
    # channel 4 looks like alpha on the example tiff
elif file_extension in {"jpg", "jpeg", "jpe", "jif", "jfif", "jfi"}:
    red_channel = 1
    green_channel = 2
    blue_channel = 3
else:
    raise Exception(f"image file extension '{file_extension}' not currently supported")

In [ ]:
# clipping image based on a shape file is not necessary for initial tests since the provided images are smaller than a single plot. 
# This section may be added later once crop data has been collected

In [ ]:
blue_raw: Optional[np.ndarray] = None
green_raw: Optional[np.ndarray] = None
red_raw: Optional[np.ndarray] = None
nir_raw: Optional[np.ndarray] = None
re_raw: Optional[np.ndarray] = None

# importing the image to be analysed
with rasterio.open(FILENAME, 'r') as raster2:
    raster2 = cast(rasterio.DatasetReader, raster2)
    band_count = cast(int, raster2.count)

    if (band_count >= blue_channel):
        blue_raw = raster2.read(blue_channel)

    if (band_count >= green_channel):
        green_raw = raster2.read(green_channel)

    if (band_count >= red_channel):
        red_raw = raster2.read(red_channel)

    if (band_count >= nir_channel):
        nir_raw = raster2.read(nir_channel)

    if (band_count >= re_channel):
        re_raw = raster2.read(re_channel)

print(band_count)

In [ ]:
# Converting digital numbers (DN) to reflectance
# This step is quite important and highly recommended, unless already have been done while image processing
# Images converted to reflectance values should range between 0 - 1 or 0 - 100%
# The equation for conversion is ((DN/(2^n)) - 1), where n is the bit size of the camera
# Digital cameras generally store images as 8 bit or 16 bit
# For this example n = 16, and thus ((DN/(2^16)) - 1) = 65535

# TODO change to optional
blue: np.ndarray
green: np.ndarray
red: np.ndarray
nir: np.ndarray
re: np.ndarray

image_max_value = 2 ** IMAGE_BITS - 1

if blue_raw is not None:
    blue = (cast(np.ndarray, blue_raw) / image_max_value).astype(float)

if green_raw is not None:
    green = (cast(np.ndarray, green_raw) / image_max_value).astype(float)

if red_raw is not None:
    red = (cast(np.ndarray, red_raw) / image_max_value).astype(float)

if nir_raw is not None:
    nir = (cast(np.ndarray, nir_raw) / image_max_value).astype(float)

if re_raw is not None:
    re = (cast(np.ndarray, re_raw) / image_max_value).astype(float)

## Vegetation Indices

In [ ]:
# This next section involves the calculation of Vegetation indices. 
# Based on this article https://en.wikipedia.org/wiki/Vegetation_index#:~:text=A%20vegetation%20index%20(VI)%20is,activity%20and%20canopy%20structural%20variations.
# A Vegetation Index is a calculation made using multispectral imagery data to highlight different qualities in vegetation such as dead plants or live leaves

# Calculating vegetation indices
# This example shows with 15 vegetation indices
# Any number of vegetation indices can be used

# Dealing with the situations division by zero
np.seterr(divide='ignore', invalid='ignore')

# Making original calculation
# Later on adjustments can be made to remove soil, or normalising the vegetation indices, etc.

# These arrays give values between 0 and 1 but can also output negative values

rgb_indices = False
nir_indices = False
re_indices = False

# RGB only
if red_raw is not None and green_raw is not None and blue_raw is not None:
    rgb_indices = True

    NGRDI_Orig = ((green).astype(float) - (red).astype(float))/((green).astype(float) + (red).astype(float))
    HUE = np.arctan((2 * (red - green - blue) )/ (30.5*(green - blue)))

    # NIR based
    if nir_raw is not None:
        nir_indices = True

        NDVI_Orig = (nir.astype(float) - red.astype(float)) / (nir.astype(float) + red.astype(float))
        GNDVI_Orig = (nir.astype(float) - green.astype(float)) / (nir.astype(float) + green.astype(float))
        ENDVI_Orig = (nir.astype(float) + green.astype(float) - 2*blue.astype(float)) / (nir.astype(float) + green.astype(float) + 2*blue.astype(float))
        SIPI_Orig = (nir.astype(float)-blue.astype(float))/(nir.astype(float) + red.astype(float))
        NLI_Orig = (((nir.astype(float))**2) - red.astype(float)) / (((nir.astype(float))**2) + red.astype(float))
        SR_Orig = nir.astype(float)/red.astype(float)
        DVI_Orig = nir.astype(float) - red.astype(float)
        RDVI_Orig = (nir.astype(float) - red.astype(float)) / ((nir.astype(float) + red.astype(float))**(1/2))

        # RE based
        if re_raw is not None:
            re_indices = True
            
            RENDVI_Orig = (re.astype(float) - red.astype(float)) / (re.astype(float) + red.astype(float))
            NDRE_Orig = (nir.astype(float) - re.astype(float)) / (nir.astype(float) + re.astype(float))
            NNIR_Orig = nir.astype(float) / (nir.astype(float) + (re.astype(float) + green.astype(float)))
            MCARI_Orig = (re.astype(float)-red.astype(float)) - 2*(re.astype(float) - green.astype(float))*(re.astype(float) / red.astype(float))
            MDD_Orig = (nir.astype(float) - re.astype(float)) - (re.astype(float) - green.astype(float))
            MARI_Orig = ((1/green.astype(float))-(1/re.astype(float)))*nir.astype(float)

In [ ]:
if nir_indices:
    # these images scale the 0 - 1 values to a 255 greyscale
    # sanity check to make sure the Vegetation Indices are producing images:
    display_image(NDVI_Orig, "VI_Test_NDVI")
    display_image(ENDVI_Orig, "VI_Test_ENDVI")
    display_image(SIPI_Orig, "VI_Test_SIPI")

## Crop Isolation

In [ ]:
# Separating crop and soil fractions based on NGRDI
# This step is optional and based on the need of the research
# For this example NGRDI has been used to classify between soil and crop, so anywhere in the image that aligns with NGRDI being negative  will be removed from the selected index
# Any vegetation indices can be used (VI_For_Classification)
# basic syntax is: 
# VI = np.where(VI_For_Classification (symbol(s): > or < or = or !=) (classification criteria), VI_Of_Interest, -math.inf)
# -math.inf is the number for null values

# these arrays contain values ranging from 0 - 1. GNDVI has been tweaked following some experimentation
# with the test data found (Test_Image_8) GNDVI seems particularly good at isolating the plants, but with a value of 0 showing the plant and 1 showing shadow
# GNDVI also seem to be best in this image for isolating or ignoring shadows. the threshold has also been adjusted based on observation
# 0.15 removes a good amount of the soil and less green crops


# RGB only
if rgb_indices:
    NGRDI = np.where(NGRDI_Orig > 0, NGRDI_Orig, -math.inf)

# NIR based
if nir_indices:
    NDVI = np.where(NGRDI_Orig > 0, NDVI_Orig, -math.inf)
    GNDVI = np.where(NGRDI_Orig > 0.15, (1-GNDVI_Orig), -math.inf)
    ENDVI = np.where(NGRDI_Orig > 0, ENDVI_Orig, -math.inf)
    SIPI = np.where(NGRDI_Orig > 0, SIPI_Orig, -math.inf)
    NLI = np.where(NGRDI_Orig > 0, NLI_Orig, -math.inf)
    SR = np.where(NGRDI_Orig > 0, SR_Orig, -math.inf)
    DVI = np.where(NGRDI_Orig > 0, DVI_Orig, -math.inf)
    RDVI = np.where(NGRDI_Orig > 0, RDVI_Orig, -math.inf)

# RE based
if re_indices:
    RENDVI = np.where(NGRDI_Orig > 0, RENDVI_Orig, -math.inf)
    NDRE = np.where(NGRDI_Orig > 0, NDRE_Orig, -math.inf)
    NNIR = np.where(NGRDI_Orig > 0, NNIR_Orig, -math.inf)
    MCARI = np.where(NGRDI_Orig > 0, MCARI_Orig, -math.inf)
    MDD = np.where(NGRDI_Orig > 0, MDD_Orig, -math.inf)
    MARI = np.where(NGRDI_Orig > 0, MARI_Orig, -math.inf)

In [ ]:
if nir_indices:
    # this image scale the 0 - 1 values to a 255 greyscale
    # sanity check to make sure the soil exclusion is working, currently no change since NGRDI_Orig already has been classified by itself:
    display_image(GNDVI, "Soil_Exclusion_Test")

In [ ]:
# # ! TEMP

if rgb_indices:
    display_image(
        np.where(
            np.repeat(np.expand_dims(NGRDI_Orig, 2), 3, axis=2) > 0,
            cv2.merge([blue, green, red]),
            0
        ),
        "temp_ndvi"
    )

## Plot Identification

In [ ]:
# FieldImageR uses manual plot isolation, equally dividing up a region specified by the user.
# For now, a similar approach will be used.

# Assumes vertical alignment (or horizontal via transpose)

img_rgb = np.squeeze(np.dstack((
    cast(np.ndarray, blue),
    cast(np.ndarray, green),
    cast(np.ndarray, red),
)))
display_image(img_rgb, "Img_RGB")

In [ ]:
plots_top_left = (435, 150)
plots_bottom_right = (860, 1250)

plots_h_range = np.arange(plots_top_left[1], plots_bottom_right[1])
plots_v_range = np.arange(plots_top_left[0], plots_bottom_right[0])

def plots_range(arr: np.ndarray):
    return arr[plots_h_range[:, None], plots_v_range[None, :]]

plots_rgb = plots_range(img_rgb)
display_image(plots_rgb, "Plot_RGB")

In [ ]:
plots_rgb_no_soil = np.where(plots_range(np.repeat(np.expand_dims(NGRDI_Orig, 2), 3, axis=2)) > 0.15, plots_rgb, np.zeros(plots_rgb.shape))
display_image(plots_rgb_no_soil, "Plot_RGB_no_soil")

## Crop Counting

In [ ]:
# TODO